# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

from hnc.hnc.PIE import Plasma_of_Ions_and_Electrons as PLASMA

from hnc.hnc.constants import *
from hnc.hnc.misc import n_from_rs, rs_from_n

from pandas import read_csv

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
                
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}

</style>
""")
import matplotlib as mpl
from cycler import cycler
mpl.rcParams['lines.linewidth'] = 2.2
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.markersize'] = 8
mpl.rcParams['font.sans-serif'] = ['Arial', 'Tahoma', 'DejaVu Sans', 'Lucida Grande', 'Verdana']
mpl.rcParams['font.size'] = 16
mpl.rcParams['axes.grid'] = True
mpl.rcParams['grid.color'] = '#636363'
mpl.rcParams['grid.alpha'] = 0.12
mpl.rcParams['grid.linewidth'] = 0.7
mpl.rcParams['grid.linestyle'] = '-'
mpl.rcParams['axes.prop_cycle'] = cycler('color',  ['#0AB34F', '#bf009f', '#00abc3', '#8e92c5', '#cc7014', '#e32851'])
mpl.rcParams['legend.fancybox'] = True
mpl.rcParams['legend.framealpha'] = 0.6
mpl.rcParams['legend.fontsize'] = 14
mpl.rcParams['figure.figsize'] = 8, 4.9443
mpl.rcParams['figure.dpi'] = 100


ModuleNotFoundError: No module named 'mendeleev'


# Function that sets up HNC QSP 

### Create and run plasma

In [2]:
name='H'
Z, A = 1,1
mi = m_p * A

ni_cc = 5.98e23
ri_cm = rs_from_n(ni_cc)
ri_AU = ri_cm*cm_to_AU


### Make βu_ii, βu_fi from using bound density from AA
If we assume the bound state does not deform at all, then we can calculate potentials based on it easily via integration

### Solving

In [3]:
from scipy.optimize import least_squares
from scipy.interpolate import interp1d,CubicSpline, PchipInterpolator

def custom_flat(hnc_matrix):
    hnc_flat = np.concatenate([hnc_matrix[0,0],hnc_matrix[0,1],hnc_matrix[1,1]])
    return hnc_flat

def custom_unflat(hnc_flat):
    hnc00, hnc01, hnc11 = np.split(hnc_flat,3)
    hnc_matrix = np.zeros((2,2,len(hnc00)))
    hnc_matrix[0,0] = hnc00
    hnc_matrix[1,0] = hnc01
    hnc_matrix[0,1] = hnc01
    hnc_matrix[1,1] = hnc11
    return hnc_matrix

def solve_least_squares(plasma, sparse_k_array, c_s_k_guess=None, ftol=1e-8):
    if c_s_k_guess is None:
        c_s_k_guess = plasma.hnc.c_s_k_matrix

    # Create functions for converting between sparse, not sparse and flattened versions of c_s_k
    get_sparse_csk = lambda c_s_k_flat: np.concatenate([PchipInterpolator(plasma.hnc.k_array, c_s_k)(sparse_k_array) for c_s_k in np.split(c_s_k_flat,3) ])
    get_csk = lambda sparse_hnc_csk: PchipInterpolator(sparse_k_array, sparse_hnc_csk)(plasma.hnc.k_array)
    get_csk_matrix_from_sparse_csk_flat =  lambda sparse_c_s_k_flat: custom_unflat(np.concatenate([get_csk(sparse_csk) for sparse_csk in np.split(sparse_c_s_k_flat,3)]))
    
    def min_function(sparse_c_s_k_flat):
        c_s_k_matrix = get_csk_matrix_from_sparse_csk_flat(sparse_c_s_k_flat)
        new_c_s_k_matrix = plasma.hnc.guess_c_s_k_matrix(c_s_k_matrix)
        plasma.hnc.c_s_k_matrix = new_c_s_k_matrix.copy()
        new_c_s_k_flat = custom_flat(new_c_s_k_matrix)
        return sparse_c_s_k_flat - get_sparse_csk(new_c_s_k_flat)
    
    c_s_k_flat_guess = custom_flat(c_s_k_guess)
    c_s_k_sparse_guess = get_sparse_csk(c_s_k_flat_guess)
    sol = least_squares(min_function, c_s_k_sparse_guess, verbose=2, ftol=ftol)#, bounds = c_s_k_bounds)
    plasma.hnc.c_s_k_matrix = get_csk_matrix_from_sparse_csk_flat(sol.x)
    plasma.hnc.set_all_matrices_from_csk(plasma.hnc.c_s_k_matrix)
    print(sol)

def best_shift(plasma, c_s_k_guess = None, ftol=1e-8, set=True):
    if c_s_k_guess is None:
        c_s_k_guess = plasma.hnc.c_s_k_matrix

    def min_function(multiplier_list):
        xii, xei, xee = multiplier_list
        multiplier_matrix = np.array([[xii,xei],[xei,xee]])
        c_s_k_matrix = multiplier_matrix[:,:,np.newaxis] * c_s_k_guess
        new_c_s_k_matrix = plasma.hnc.guess_c_s_k_matrix(c_s_k_matrix)
        
        return c_s_k_matrix.flatten() - new_c_s_k_matrix.flatten()
    
    sol = least_squares(min_function, np.ones((3)) , verbose=2, ftol=ftol)#, bounds = c_s_k_bounds)
    xii, xei, xee = sol.x
    multiplier_matrix = np.array([[xii,xei],[xei,xee]])
    
    plasma.hnc.c_s_k_matrix = multiplier_matrix[:,:,np.newaxis] * c_s_k_guess
    print(sol)
    return sol

In [4]:
def plasma_f(T_eV, Rc, Zbar, closure='svt'):
    plasma = PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Zbar, find_βuee=False,
             βu_options = {'add_bridge':True,'bridge':'yukawa', 'pseudopotential':False}, 
             qsp_options = {'Te_c_type':'DMC'},
             hnc_options={'kappa_multiscale':10, 'R_max':10,'N_bins':1000,'oz_method':closure},
             hnc_solve_options={'num_iterations':10000,'alpha_Picard':0.1, 'verbose':False,'iters_to_wait':500},
             root_options={'options':{'maxfev':100,'xtol':1e-14}})

    return plasma

In [5]:
def ocp_f(T_eV, Zbar):
    ocp = PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Zbar, find_βuee=False,
             βu_options = {'add_bridge':True,'bridge':'yukawa', 'pseudopotential':False}, 
                 hnc_options={'kappa_multiscale':10, 'R_max':10,'N_bins':1000,'oz_method':'standard'},
                 hnc_solve_options={'num_iterations':10000,'alpha_Picard':0.1, 'verbose':False,'iters_to_wait':500},
                 root_options={'options':{'maxfev':100,'xtol':1e-14}})
    ocp.run_ocp_hnc()
    return ocp

def yp_f(T_eV, Zbar):
    yukawa_plasma =  PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Zbar, find_βuee=False,
                 βu_options = {'add_bridge':True,'bridge':'yukawa', 'pseudopotential':False}, 
                 hnc_options={'kappa_multiscale':10, 'R_max':10,'N_bins':1000,'oz_method':'standard'},
                 hnc_solve_options={'num_iterations':10000,'alpha_Picard':0.1, 'verbose':False,'iters_to_wait':500},
                 root_options={'options':{'maxfev':100,'xtol':1e-14}})
    yukawa_plasma.run_yukawa_hnc();
    return yukawa_plasma

In [6]:
def save_plasma(plasma, Rc):
    T_eV = plasma.Ti_eV
    Zbar = plasma.qsp.Zbar
    ocp = ocp_f( T_eV, Zbar)
    yp  = yp_f(T_eV, Zbar )
    
    # Make header
    info_header = (f"# Zach Johnson Dec/11/2023 CHNC for TCCW Paper" + '\n' + 
                   f"# Ti={T_eV:0.3e}[eV], Tie={plasma.qsp.Tie_c*AU_to_eV:0.3e}[eV] , Te_c={plasma.qsp.Te_c*AU_to_eV:0.3e}[eV], n_i={plasma.ni_cc:0.3e}[1/cc], n_e={Zbar*plasma.ni_cc:0.3e}[1/cc]" + '\n' + 
                   f"# ri={ri_AU:0.3f}, Zbar={Zbar:0.3f}, Rc={Rc:0.3f}[AU], Γii={plasma.qsp.Γii:0.3e}, Γei={plasma.qsp.Γei:0.3e} , Γee={plasma.qsp.Γee:0.3e}, κ={plasma.qsp.get_κ():0.3e}" 
                  )
    column_header = f"{'r/ri':10}  {'g_ii':10}  {'g_ei':10}  {'g_ee':10}  {'OCP:g_ii':10}  {'Yuk:g_ii':10}  {'βu_ii_eff':10}  {'βu_OCP':10}  {'βu_Yuk':10}"
    header = info_header + '\n' + column_header
    
    # Make save array
    save_array = np.array([  plasma.hnc.r_array, plasma.hnc.h_r_matrix[0,0]+1, plasma.hnc.h_r_matrix[0,1]+1, plasma.hnc.h_r_matrix[1,1]+1,
                           ocp.ocp_hnc.h_r_matrix[0,0]+1, yp.yuk_hnc.h_r_matrix[0,0] + 1, plasma.βueff_r_matrix_with_B[0,0],
                           ocp.ocp_hnc.βu_r_matrix[0,0], yp.yuk_hnc.βu_r_matrix[0,0]]).T
    
    np.savetxt(f"/home/zach/plasma/hnc/TCCW/Final_TCCW/results/CHNC_H_rs{ri_AU:0.3f}_TeV{plasma.Ti_eV:0.3f}.dat", save_array, 
               header=header, fmt='%10.5e', comments='')

### Now Load Data from George
Rc is interpreted as the average bound radius, in AU in the file. Empty-core Ashcroft assumed.

In [7]:
GP_data   = read_csv("/home/zach/plasma/hnc/TCCW/Final_TCCW/GP_data/H_combined.dat", header=0, delim_whitespace=True)
T_eV_list = GP_data['Te(eV)']
Rc_list   = GP_data['Rc'] # AU
Zbar_list = GP_data['Z_bar'] #Average atom Zbar GP_data
GP_data

,Te(eV),rho/rho0,Ne_b,Ne_quasi,Ne_sc,Z_bar,Rc,mu,nu_mom,nu_tot,sigma,k_th,L,G
0,0.1,1.0,0.0,0.0,1.0,1.0,0,0.60830,0.112600,0.44460,0.3495,0.004226,3.290,1.235000e-05
1,0.2,1.0,0.0,0.0,1.0,1.0,0,0.60960,0.111800,0.44000,0.3526,0.008530,3.291,1.231000e-05
2,0.5,1.0,0.0,0.0,1.0,1.0,0,0.61320,0.109700,0.42720,0.3618,0.021930,3.298,1.221000e-05
3,1.0,1.0,0.0,0.0,1.0,1.0,0,0.61840,0.106800,0.40890,0.3765,0.045970,3.323,1.207000e-05
4,2.0,1.0,0.0,0.0,1.0,1.0,0,0.62610,0.102800,0.38020,0.4044,0.101200,3.404,1.190000e-05
5,5.0,1.0,0.0,0.0,1.0,1.0,0,0.62760,0.099500,0.32790,0.4783,0.311000,3.539,1.198000e-05
6,10.0,1.0,0.0,0.0,1.0,1.0,0,0.54060,0.101800,0.28640,0.5767,0.821000,3.874,1.215000e-05
7,20.0,1.0,0.0,0.0,1.0,1.0,0,0.09214,0.103100,0.24640,0.8232,2.499000,4.130,1.116000e-05
8,50.0,1.0,0.0,0.0,1.0,1.0,0,-2.42900,0.088140,0.19200,1.9690,13.760000,3.803,7.608000e-06
9,100.0,1.0,0.0,0.0,1.0,1.0,0,-8.66900,0.063270,0.14550,4.4600,61.560000,3.756,4.457000e-06


In [8]:
plasma_list = []

for T_eV, Rc, Zbar in zip(T_eV_list, Rc_list, Zbar_list):
    plasma = plasma_f(T_eV, Rc/ri_AU, Zbar )
    # plasma = plasma_f(T_eV, Rc/ri_AU, None )# TF
    plasma_list.append(plasma)

Te_in_eV: 0.100
Ti_in_eV: 0.100
Te_in_eV: 0.200
Ti_in_eV: 0.200
Te_in_eV: 0.500
Ti_in_eV: 0.500
Te_in_eV: 1.000
Ti_in_eV: 1.000
Te_in_eV: 2.000
Ti_in_eV: 2.000
Te_in_eV: 5.000
Ti_in_eV: 5.000
Te_in_eV: 10.000
Ti_in_eV: 10.000
Te_in_eV: 20.000
Ti_in_eV: 20.000
Te_in_eV: 50.000
Ti_in_eV: 50.000
Te_in_eV: 100.000
Ti_in_eV: 100.000
Te_in_eV: 200.000
Ti_in_eV: 200.000
Te_in_eV: 500.000
Ti_in_eV: 500.000
Te_in_eV: 1000.000
Ti_in_eV: 1000.000
Te_in_eV: 2000.000
Ti_in_eV: 2000.000


In [20]:
guess_c_s_k_matrix = None
for Rc, plasma in zip(Rc_list[4:5], plasma_list[4:5]): 
    # if T_eV == 2.0:
    plasma.run_hnc(c_s_k_guess = guess_c_s_k_matrix)
    if plasma.hnc.tot_err_list[-1] > 1e-7:
        best_shift(plasma)
        solve_least_squares(plasma, np.linspace(plasma.hnc.k_array[0], plasma.hnc.k_array[-1], num=200))
        solve_least_squares(plasma, np.linspace(plasma.hnc.k_array[0], plasma.hnc.k_array[-1], num=400))
        save_plasma(plasma, Rc)
    else:
        save_plasma(plasma, Rc)
    # guess_c_s_k_matrix = plasma.hnc.c_s_k_matrix.copy()

0: Initial condition Total Error: 2.621e+00
Starting Ng loop, using best index so far:  499
Exiting status 0, reverting to best index so far: 499
Final iter:502, Total Error: 6.321e-12
Te_in_eV: 2.000
Ti_in_eV: 2.000
0: Initial condition Total Error: 5.246e+00
Starting Ng loop, using best index so far:  499
Exiting status 0, reverting to best index so far: 501
Final iter:504, Total Error: 1.836e-13
Te_in_eV: 2.000
Ti_in_eV: 2.000
0: Initial condition Total Error: 3.490e+01
Exiting status 0, reverting to best index so far: 435
Final iter:436, Total Error: 9.941e-09


### Plot and Comparison to QMD

In [ ]:

# for plasma, Rc in zip(plasma_list[4:5], Rc_list[4:5]):    
#     save_plasma(plasma, Rc)

## Specific Heat

In [13]:
plasma_list[-1].run_hnc()

0: Initial condition Total Error: 1.960e-02
Exiting status 0, reverting to best index so far: 138
Final iter:139, Total Error: 9.513e-09


In [14]:
plasma_list[-1].get_effective_ion_cp()

0: Initial condition Total Error: 3.037e-02
Exiting status 0, reverting to best index so far: 142
Final iter:143, Total Error: 9.658e-09
0: Initial condition Total Error: 2.925e-02
Exiting status 0, reverting to best index so far: 142
Final iter:143, Total Error: 9.303e-09
0: Initial condition Total Error: 3.075e-02
Exiting status 0, reverting to best index so far: 142
Final iter:143, Total Error: 9.780e-09
0: Initial condition Total Error: 2.962e-02
Exiting status 0, reverting to best index so far: 142
Final iter:143, Total Error: 9.420e-09

_____________________________
Heat Capacity Results (Effective Ion Picture) 
c_p = 2.057e+07 [J/m^3/K] = 2.057e+08 [erg/cc/K]
c_p^ideal = 2.064e+07 [J/m^3/K] = 2.064e+08 [erg/cc/K]
c_v = 1.239e+07 [J/m^3/K] = 1.239e+08 [erg/cc/K]

γ = cp/cv = 1.661e+00

E/nkBT = 1.825, P/nkBT = 1.154 
cp/nkB = 2.492, cv/nkB = 1.500 

Total cv/nkB estimate (add ideal electrons):
c_v_tot_estimate = 1.500


In [16]:
plasma_list[-1].get_effective_ion_cp(ε_derivative=1e-6, ε_table=1e-16, N_table=2)

0: Initial condition Total Error: 2.999e-02
Exiting status 0, reverting to best index so far: 142
Final iter:143, Total Error: 9.538e-09
0: Initial condition Total Error: 2.999e-02


KeyboardInterrupt: 

In [ ]:
plasma_list[-1].get_effective_ion_cp(ε_derivative=1e-8, ε_table=1e-6, N_table=2)

In [ ]:
plasma_list[-1].get_effective_ion_cp(ε_derivative=1e-6, ε_table=1e-4, N_table=3)

## Conductivity from Ziman

https://arxiv.org/pdf/1508.06101.pdf eq.5 in Hansen & McDonald the eqn's 10.2.4, 10.2.11

$$
\sigma^{-1} = \rho= \frac{1}{32 \sqrt{\pi} Z} \frac{m_e^2}{(2 m_e T_e^c)^{3/2}} \int^{\infty}_0 e^{-q^2/(8 m_e T_e^c)}  |\tilde{V}(q)|^2 S_{ii}(q) q^3 dq
$$
where the screened potential 
$$\tilde{V}(q) = V(q)/\epsilon,$$
for $$\frac{1}{\epsilon}  = 1 + \frac{4 \pi}{k^2}\chi_{ee}(k) $$ 
and 
$$\chi_{ee}(k) = -\beta_e^c n_e S_{ee}(k) $$

In [ ]:
from scipy.integrate import simpson

def conductivity(plasma):
    q_array = plasma.hnc.k_array/plasma.qsp.ri 
    
    rhosum = np.sum(plasma.hnc.rho/plasma.qsp.ri**3)
    x = plasma.hnc.rho/plasma.qsp.ri**3/rhosum
    S_matrix = np.diag(x)[:,:,np.newaxis] + rhosum * x[:,np.newaxis,np.newaxis]*x[np.newaxis,:,np.newaxis]*plasma.hnc.h_k_matrix
    χ_ee = -plasma.qsp.βe_c*plasma.qsp.ne*S_matrix[1,1]
    kTF = plasma.qsp.get_κ()/plasma.qsp.ri
    # χ_ee = -q_array**2/(4*π)* kTF**2/(kTF**2 + q_array**2)
    ε = 1/( 1 + 4*π/q_array**2 * χ_ee)
    Vtilde = -plasma.Zbar/(plasma.hnc.r_array*plasma.qsp.ri)/ε
    # print(ε)
    integrand = np.exp(-plasma.qsp.βe_c * q_array**2/8) * np.abs(Vtilde)**2 * S_matrix[0,0] * q_array**3
    ρ_AU = 1/(  32*np.sqrt(π)*plasma.Zbar*(2*plasma.qsp.Te_c)**1.5) * simpson(integrand, q_array)  
    σ_AU = 1/ρ_AU
    σ_Spercm = σ_AU * AU_to_Siemens/AU_to_cm
    return σ_Spercm
print(f"Conductivity: σ = {conductivity(plasma):0.3e}")

In [ ]:
fig, ax = plt.subplots( facecolor='white')

ax.plot(T_eV_list, [conductivity(plasma) for plasma in plasma_list], label='Ziman')

ax.set_ylabel(r"$\sigma $[S/cm]")
ax.set_xlabel(r"$T$ [eV]")


# Inset Text Box
# props = dict(boxstyle='round', facecolor='white', alpha=0.5)
# inset_text = f"H\n {sci_notation(ni_cc,3)} [cc$^{{-1}}$]" 
# ax.text(0.35, 0.95, inset_text, transform=ax.transAxes, fontsize=14,
#         verticalalignment='top', bbox=props)

# ax.errorbar([tccw_TeV],[tccw_η_cgs], yerr=[tccw_η_cgs_err], barsabove=True, capsize=5, marker='.', label='MD Simulation')

# Scales
ax.set_yscale('log')
ax.set_xscale('log')
# ax.set_ylim(5e4,5e6)

# Legend
ax.legend()


## Born Green Viscocity
See [Morioka 2004](https://www.sciencedirect.com/science/article/pii/S0022309304003692)

\begin{align}
η = f_{BG} \frac{2 \pi}{15} \left(\frac{M}{RT} \right)^{1/2} \frac{n^2}{N_a} \int^\infty_0 g(r) \frac{d\phi}{dr}r^4 dr
\end{align}
Resulting units is Pa s


In [ ]:
from scipy.integrate import simpson

def get_ηBG(plasma):
    plasma.hnc.invert_HNC_OZ([1])
    fBG = -1
    n_AU = n_from_rs(plasma.qsp.ri)
    r_array = plasma.hnc.r_array*plasma.qsp.ri
    gii = plasma.hnc.h_r_matrix[0,0]+1
    dφdr = plasma.qsp.Ti*np.gradient(plasma.hnc.βueff_r_matrix[0,0], r_array)

    integrand = r_array**4 * gii * dφdr
    ##
    fig, ax = plt.subplots()
    ax.plot(r_array, dφdr*r_array**4)
    ###
    integral = simpson(integrand, x=r_array)
    η = fBG* 2*π/15 * (plasma.qsp.βi*plasma.qsp.m_i)**0.5*n_AU**2*integral
    return η

η_SI = get_ηBG(plasma_min)*AU_to_Pa*AU_to_s # Pa s = kg m^-1 s^-1 
# print(f"SI: η = {η_SI:0.3e} [kg m^-1 s^-1] (or [Pa s])")
# η_cgs = η_SI * 1000/100 # g cm^-1 s^-1
# print(f"CGS: η = {η_cgs:0.3e} [g cm^-1 s^-1]")
ηCHNC_minT_mPas = η_SI*1000
print(f"CGS: η = {ηCHNC_minT_mPas:0.3e} [m Pa s]")

Smith_lowT_η

In [ ]:
η_plasma_list = []
for T_eV, Rc, Zbar in zip(GP_Al_T_few[2:], GP_Al_Rc[2:], GP_Al_Z_bar[2:]):
    print(T_eV, Rc, Zbar)
    tmp_plasma = plasma_f(T_eV, Rc, Zbar, closure='svt')
    tmp_plasma.run_hnc()
    η_plasma_list.append(tmp_plasma)

In [ ]:
ηCHNC_unnormalized_mPaS_list = []
for plasma in η_plasma_list:
    ηCHNC_unnormalized_mPaS_list.append( get_ηBG(plasma)*AU_to_Pa*AU_to_s*1000  )

In [ ]:
(Smith_lowT_η/ηCHNC_minT_mPas)

In [ ]:
TCHNC_eV_final = np.concatenate([[Smith_lowT_TeV] ,GP_Al_T_few[2:]])
ηCHNC_normalized_mPaS_list = np.array(ηCHNC_unnormalized_mPaS_list)* (Smith_lowT_η/ηCHNC_minT_mPas)

ηCHNC_mPas_unnormalized_final = np.concatenate([[ηCHNC_minT_mPas] , ηCHNC_unnormalized_mPaS_list ])
ηCHNC_mPas_normalized_final = np.concatenate([[Smith_lowT_η] , ηCHNC_normalized_mPaS_list ])

np.savetxt("/home/zach/plasma/hnc/TCCW/Final_TCCW/results/Al_η_MPas_Final.dat", np.array([TCHNC_eV_final, ηCHNC_mPas_unnormalized_final, ηCHNC_mPas_normalized_final ]).T, 
           header="T[eV]   η[mPas](unnormalized)  η[mPas](Smith-normalized) ", comments='')
# /home/zach/plasma/hnc/TCCW/Final_TCCW/results/η_cgs_Al_Final.dat

## Ziman Electron-Ion Relaxation Time

See Y. V. Petrov [Electron–ion energy exchange in simple metals in
Ziman approach](https://iopscience.iop.org/article/10.1088/1742-6596/1556/1/012005/pdf)